In [1]:
import torch
import torch.nn as nn
from transformer import Transformer
from datasets import load_dataset
#import spacy
from transformers import BertTokenizerFast
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import torch.optim as optim

In [2]:
torch.cuda.is_available()

True

In [3]:
#torch.Tensor([["a"],["b"],["c"]])

In [4]:
english_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", padding_side = "right", truncation_side = "right", model_max_length = 64)
german_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", padding_side = "right", truncation_side = "right", model_max_length = 64)

C:\Users\Ionut Anghelina\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
print("English Tokenizer Special Tokens:", english_tokenizer.special_tokens_map)
print("German Tokenizer Special Tokens:", german_tokenizer.special_tokens_map)

English Tokenizer Special Tokens: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51

In [6]:
special_tokens = {"bos_token": "<s>"}

# Add the special token to the tokenizer
german_tokenizer.add_special_tokens(special_tokens)

# Check the updated special tokens and see if <s> has been added
print(german_tokenizer.special_tokens_map)

# Get the ID of the newly added <s> token
bos_token_id = german_tokenizer.convert_tokens_to_ids("<s>")
#print(f"<s> token ID: {bos_token_id}")

# Get the token for ID 2 (which could be <s> or another token)
#token_for_id_2 = german_tokenizer.convert_ids_to_tokens(32100)
#print(token_for_id_2)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id

In [7]:
special_tokens = {"bos_token": "<s>"}

english_tokenizer.add_special_tokens(special_tokens)

print(english_tokenizer.special_tokens_map)


bos_token_id = english_tokenizer.convert_tokens_to_ids("<s>")
#print(f"<s> token ID: {bos_token_id}")

#token_for_id_2 = english_tokenizer.convert_ids_to_tokens(32100)
#print(token_for_id_2)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id

In [8]:
start_token_id = english_tokenizer.convert_tokens_to_ids('<s>')
end_token_id = english_tokenizer.convert_tokens_to_ids('</s>')

print(f"<s> token ID: {start_token_id}")
print(f"</s> token ID: {end_token_id}")

<s> token ID: 32100
</s> token ID: 1


In [9]:
start_token_id = german_tokenizer.convert_tokens_to_ids('<s>')
end_token_id = german_tokenizer.convert_tokens_to_ids('</s>')

print(f"<s> token ID: {start_token_id}")
print(f"</s> token ID: {end_token_id}")

<s> token ID: 32100
</s> token ID: 1


In [10]:
source_vocab_dim = english_tokenizer.vocab_size + 1
target_vocab_dim = german_tokenizer.vocab_size + 1

In [11]:
dataset = load_dataset("wmt/wmt14","de-en")

In [12]:
print([x['en'] for x in dataset["train"][:10]['translation']])

['Resumption of the session', 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.', "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'You have requested a debate on this subject in the course of the next few days, during this part-session.', "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.", "Please rise, then, for this minute' s silence.", "(The House rose and observed a minute' s silence)", 'Madam President, on a point of order.', 'You will be aware from the press and television that there have been a num

In [13]:
class DatasetClass(Dataset):
    
    def __init__(self, dataset, partition):
        
        
        self.dataset = dataset[partition]
        self.english_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", padding_side = "right", truncation_side = "right", model_max_length = 64)
        self.german_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base", padding_side = "right", truncation_side = "right", model_max_length = 64)
        
        if os.path.exists(f"./SavedData/{partition}"):
            self.english_texts = torch.load(f"./SavedData/{partition}/en.pth")
            self.german_texts = torch.load(f"./SavedData/{partition}/de.pth")
        else:
            
            self.english_texts = [self.applyEnglishTokenizer(sample['en']).unsqueeze(0) for sample in tqdm(self.dataset[:]['translation'])]
            self.german_texts = [self.applyGermanTokenizer(sample['de']).unsqueeze(0) for sample in tqdm(self.dataset[:]['translation'])]
            
            self.english_texts = torch.cat(self.english_texts)
            self.german_texts = torch.cat(self.german_texts)
            
            print(self.english_texts.shape)
            
            if not os.path.exists("./SavedData"):
                os.mkdir("./SavedData")
            if not os.path.exists(f"./SavedData/{partition}"):
                os.mkdir(f"./SavedData/{partition}")
            
            torch.save(self.english_texts,f"./SavedData/{partition}/en.pth")
            torch.save(self.german_texts,f"./SavedData/{partition}/de.pth")
            
        
    def applyEnglishTokenizer(self, text):
        #print(text)
        tok = self.english_tokenizer(text, padding = "max_length", truncation = "only_first", add_special_tokens=True,
               return_tensors = "pt")['input_ids'].squeeze(0)
    
        bos_token = torch.Tensor([32100])
        tok = torch.cat([bos_token, tok])
        
        #print(tok.shape)
        if tok[-1] == 1:
            return torch.cat([tok[:-2], tok[-1:]])
        else:
            return tok[:-1]
            
    def applyGermanTokenizer(self, text):
        tok = self.german_tokenizer(text, padding = "max_length", truncation = "only_first", add_special_tokens=True,
               return_tensors = "pt")['input_ids'].squeeze(0)
    
        bos_token = torch.Tensor([32100])
        tok = torch.cat([bos_token, tok])
        
        if tok[-1] == 1:
            return torch.cat([tok[:-2], tok[-1:]])
        else:
            return tok[:-1]
        
    def __getitem__(self, idx):
        
        return self.english_texts[idx], self.german_texts[idx]
        
    def __len__(self):
        
        return len(self.english_texts)
        

In [14]:
train_dataset = DatasetClass(dataset, "train")
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle = True)

C:\Users\Ionut Anghelina\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
 26%|█████████████████▋                                                   | 1159264/4508785 [07:48<2:18:49, 402.12it/s]

KeyboardInterrupt: 

In [ ]:
valid_dataset = DatasetClass(dataset, "validation")
valid_dataloader = DataLoader(valid_dataset, batch_size = 1, shuffle = False)

In [ ]:
model = Transformer(vocab_size = source_vocab_dim, target_vocab_size = target_vocab_dim, source_seq_len = 64, target_seq_len = 64, embedding_size = 64, no_encoder_blocks = 2, no_decoder_blocks = 2)

In [ ]:
#model.load_state_dict(torch.load("./checkpoints/model_parameters.pth"))

In [ ]:
#target_vocab_dim

In [ ]:
model.cuda()

In [ ]:
#print(train_dataset.english_texts.shape)

In [ ]:
optimizer = optim.Adam(model.parameters(),lr = 5e-4)


In [ ]:
loss_fn = nn.CrossEntropyLoss(ignore_index = 0)

In [ ]:
#model.load_state_dict(torch.load('checkpoints/model_parameters.pth'))

In [ ]:
EPOCHS = 10

In [ ]:
for e in range(EPOCHS):
    train_loss = 0
    
    for i, batch in tqdm(enumerate(train_dataloader)):

        
        
    
        model.train()
        optimizer.zero_grad()

        english_batch, german_batch = batch
        english_batch = english_batch.long().cuda()
        german_batch = german_batch.long().cuda()
        
      


        pred = model(english_batch, german_batch)
        pred = pred.view(-1, target_vocab_dim)
        batch_size = english_batch.shape[0]
        #print(pred.shape)
        target = german_batch[:, 1:]
        
        #print(pred.shape)
        added = torch.zeros(batch_size, 1).cuda()
 
        target = torch.cat([target, added], dim = 1)

        target = target.view(-1).long()

        #print(german_batch)
        #print(pred) 
        #print(target)
        loss = loss_fn(pred, target)

        train_loss += loss.item()

        loss.backward()

        optimizer.step()
        
        torch.save(model.state_dict(), f'checkpoints/model_parameters_2M_{e}.pth')

    print(f"The train loss on epoch {e} is {train_loss}")

In [ ]:
#model.load_state_dict(torch.load('checkpoints/model_parameters_2M_3.pth'))

In [ ]:
# import os
# os.mkdir("checkpoints")

# torch.save(model.state_dict(), 'checkpoints/model_parameters.pth')

In [ ]:
def translate(text):
    x = DatasetClass(dataset, "test")
    english_text = x.applyEnglishTokenizer(text).unsqueeze(0).long().cuda()
    #print(model.decode(64,english_text))
    t = list(model.decode(64, english_text)[0].cpu().numpy())
    print(t)
    print([y for y in t])
    return [x.english_tokenizer.convert_ids_to_tokens(int(y)) for y in t]

In [ ]:
print(translate("yOU ARE"))

In [ ]:
german_tokenizer.convert_tokens_to_ids("<s>")

In [ ]:
german_tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'})

print(german_tokenizer.special_tokens_map)
print(german_tokenizer.convert_ids_to_tokens(2))

In [ ]:
bos_token_id = english_tokenizer.convert_tokens_to_ids('<s>')
eos_token_id = english_tokenizer.convert_tokens_to_ids('</s>')
print(bos_token_id, eos_token_id)